<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

Regresssion with scikit-learn<br><br> using Soccer Dataset

<br><br></p>


We will again be using the open dataset from the popular site <a href="https://www.kaggle.com">Kaggle</a> that we used in Week 1 for our example. 

Recall that this <a href="https://www.kaggle.com/hugomathien/soccer">European Soccer Database</a> has more than 25,000 matches and more than 10,000 players for European professional soccer seasons from 2008 to 2016. 

**Note:** Please download the file *database.sqlite* if you don't yet have it in your *Week-7-MachineLearning* folder.

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Import Libraries<br><br></p>


In [1]:
import sqlite3
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Read Data from the Database into pandas
<br><br></p>


In [2]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [3]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [4]:
df.shape

(183978, 42)

In [5]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Declare the Columns You Want to Use as Features
<br><br></p>


So, now we'll select some of them.
We'll select the ones we want to use
for predicting the overall rating.

We won't select overall rating since that will be our target for the regression analysis

In [6]:
features = [
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

In [7]:
type(features)

list

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Specify the Prediction Target
<br><br></p>


In [8]:
target = ['overall_rating']

In [9]:
type(target)

list

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Clean the Data<br><br></p>


In [10]:
df = df.dropna()

In [11]:
df.shape

(180354, 42)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Extract Features and Target ('overall_rating') Values into Separate Dataframes
<br><br></p>


Let's now create the two data frames.
Like you're used to it now, we need an x, the input,
and a y to go from, with the analogy again,
of y=f(x), with features and target values.
X will be our input data frame.

In [12]:
X = df[features]

In [14]:
type(X)

pandas.core.frame.DataFrame

In [13]:
y = df[target]

In [15]:
type(y)

pandas.core.frame.DataFrame

Let us look at a typical row from our features: 

In [16]:
X.iloc[2]

potential             66.0
crossing              49.0
finishing             44.0
heading_accuracy      71.0
short_passing         61.0
volleys               44.0
dribbling             51.0
curve                 45.0
free_kick_accuracy    39.0
long_passing          64.0
ball_control          49.0
acceleration          60.0
sprint_speed          64.0
agility               59.0
reactions             47.0
balance               65.0
shot_power            55.0
jumping               58.0
stamina               54.0
strength              76.0
long_shots            35.0
aggression            63.0
interceptions         41.0
positioning           45.0
vision                54.0
penalties             48.0
marking               65.0
standing_tackle       66.0
sliding_tackle        69.0
gk_diving              6.0
gk_handling           11.0
gk_kicking            10.0
gk_positioning         8.0
gk_reflexes            8.0
Name: 2, dtype: float64

Let us also display our target values: 

In [18]:
y.head()

,overall_rating
0,67.0
1,67.0
2,62.0
3,61.0
4,61.0


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Split the Dataset into Training and Test Datasets
<br><br></p>


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(1) Linear Regression: Fit a model to the training set
<br><br></p>


We'll perform two different modeling operations
using different regression techniques.
First, we will use a linear regressor.


So, using the fit method of the regressor,
we fine-tuned the parameters of the linear regressor
to capture the interactions between the two sets,
the x_train and y_train.
So, we are trying to fit
x_train and y_train and create a model.
We can then use this predict method,
because we now have a model,
a linear-regression-based model.
We'll use the predict method of that trained model
to perform the prediction on the test set,
which is our x_test.

In [20]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Perform Prediction using Linear Regression Model
<br><br></p>


As a reminder,
note that the model has never seen any sample
from this test set.
So it's predicting on a new data set.

In [21]:
y_prediction = regressor.predict(X_test)
y_prediction

array([[ 66.51284879],
       [ 79.77234615],
       [ 66.57371825],
       ..., 
       [ 69.23780133],
       [ 64.58351696],
       [ 73.6881185 ]])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

What is the mean of the expected target value in test set ?
<br><br></p>


In [22]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluate Linear Regression Accuracy using Root Mean Square Error

<br><br></p>


We use, actually, as we describe Root Mean Square Error
to measure the prediction accuracy of our regressor.
That's the RMSE, Root Mean Square Error, in short.
So RMSE captures the variation of the predicted value,
as you would remember, from the observed value.
So, an RMSE score of zero
means perfect prediction with no errors,
which is the ideal scenario, which almost never happens.
When comparing two regression models,
then the one with the smaller RMSE will be better
since its predictions will have smaller difference
from the observed values, or the measured values, before.

In [23]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [29]:
type(RMSE)

float

So when we compute RMSE
and print it out,
we see that the linear model gives us an RMSE of 2.8.
This is a good start.
Since the range of overall rating is from 33 to 94
with a mean of, as you would remember, approximately 68.

In [24]:
print(RMSE)

2.80530304685521


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(2) Decision Tree Regressor: Fit a new regression model to the training set
<br><br></p>


Let's see if we can improve upon the prediction accuracy now
by using a slightly more complex model.
That would be, let's say, a decision tree regressor.
A decision tree regressor
builds a model in a top-down manner
by splitting data set on an attribute.
So the algorithm chooses the attribute
which gives maximum reduction in standard deviation.
You'll here more about these
in your machine learning course.


So now, let's use the decision tree regressor
to capture player performance as a function of,
again, their attributes.
The fifth method performs the fine-tuning again.
So we are doing the exact same thing.
We have a regressor,
but, since this is a tree, it has a depth.
The maximum depth is 20.
And we are using that regressor
to fit the training and
input and output training data sets, so x_train and y_train.
We changed the method, but the fifth line stays the same.
The regressor will be just of another class
and another method.

In [25]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Perform Prediction using Decision Tree Regressor
<br><br></p>


In [26]:
y_prediction = regressor.predict(X_test)
y_prediction

array([ 62.        ,  84.        ,  62.38666667, ...,  71.        ,
        62.        ,  72.        ])

In [28]:
type(y_prediction)

numpy.ndarray

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

For comparision: What is the mean of the expected target value in test set ?
<br><br></p>


In [27]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluate Decision Tree Regression Accuracy using Root Mean Square Error

<br><br></p>


To get an idea of the RMSE,
we know that a Root Mean Square Error of 100,
for example, would be too high because our mean is 68.
And our RMSE is higher than our mean value.

In [30]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [31]:
print(RMSE)

1.4595096234841023


Again, the RMSE captures variance of the predicted value
from the actual value by our system.
So, it is a measure of how
model performs against operations.
So I'd say
an RMSE of 1.44 for a test set
that has a mean of 68 for the target variable
is pretty good.
And the model never got a chance to look at the test set
before the prediction,
so that ensures our evaluation was
on a data set
that the model hasn't seen.